In [36]:
import torch
from collections import OrderedDict, defaultdict
from collections import deque, defaultdict
dev=torch.device('cpu')
centroids=torch.load('/home/jzgrp/xiaomenghan/vptq_qwen2-7/model.layers.0.self_attn.q_proj_centroids.pt').to(dev)
index=torch.load('/home/jzgrp/xiaomenghan/vptq_qwen2-7/model.layers.0.self_attn.q_proj_indices.pt').to(dev)
print(index.shape)
centroids=centroids.weight.view(1,65536,8)
high_mask = 0xffff0000  # 未使用，可保留或删除
low_mask = 0x0000ffff   # 未使用，可保留或删除
new_index = torch.zeros([448, 3584], dtype=torch.int32, device=dev)
lower = index[0] & 0xFFFF              # 提取低16位，形状 (448, 1792)
upper = (index[0] >> 16) & 0xFFFF      # 提取高16位，形状 (448, 1792)
new_index = torch.stack([lower, upper], dim=2).view(448, 3584)  # 重组为 (448, 3584)
centroids = centroids.view(65536, 8).to(dev)  # 修正拼写错误并准备张量
recon_weight = torch.zeros([448, 3584], device=dev)
assert new_index.min() >= 0 and new_index.max() < 65536, "new_index 包含越界索引"
# 使用矢量化操作替代嵌套循环
recon_weight = centroids[new_index, 0]
print(recon_weight.shape)
centroids = centroids.view(16384, 32).to(device=dev, dtype=torch.float32)
# 创建 code 张量，确保在 dev 上并为 float32
code = torch.zeros([16384, 4], dtype=torch.float32, device=dev)
code = centroids[:, [0, 8, 16, 24]]

torch.Size([1, 448, 1792])
torch.Size([448, 3584])


/tmp/ipykernel_3499129/2134733029.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  centroids=torch.load('/home/jzgrp/xiaomenghan/vptq_qwen2-7/model.layers.0.self_attn.q_p

In [20]:
# def split_tensor(tensor,subrows,subcoloums):
#     # 获取张量的形状
#     height, width = tensor.shape
    
#     # 计算可以分割的块数
#     num_blocks_height = height // subrows
#     num_blocks_width = width // subcoloums
    
#     # 定义一个函数来处理每个块
#     def process_block(block):
#         # 获取块的形状
#         block_height, block_width = block.shape
#         # 创建一个 16x16 的张量，填充 -1
#         padded_block = torch.full((subrows, subcoloums), -1, dtype=block.dtype)
#         # 将原始块复制到填充的张量中
#         padded_block[:block_height, :block_width] = block
#         return padded_block
    
#     # 创建一个空列表来存储分割后的块
#     split_blocks = []
    
#     # 遍历每个块
#     for i in range(num_blocks_height ):
#         for j in range(num_blocks_width ):
#             # 计算块的起始和结束索引
#             start_row = i * subrows
#             end_row = (i + 1) * subrows
#             start_col = j * subcoloums
#             end_col = (j + 1) * subcoloums
            
#             # 获取块
#             block = tensor[start_row:end_row, start_col:end_col]
            
#             # 处理块并添加到列表中
#             split_blocks.append(process_block(block))
    
#     return split_blocks
# split_blocks = split_tensor(new_index,2,16)

# # 打印分割后的块数量
# print(f"Number of split blocks: {len(split_blocks)}")

# # 打印第一个块的形状
# print(f"Shape of first block: {split_blocks[0].shape}")

In [33]:
class PE:
    def __init__(self, read_ready,index,subrows,subcoloums,spad):
        self.read_ready=read_ready
        self.index=index
        self.subrows=subrows
        self.subcoloums=subcoloums
        self.rowlast=None
        self.last=None
        self.cycle=None
        self.spad=spad
    def split_tensor(self):
    # 获取张量的形状
        height, width = self.index.shape
        # 计算可以分割的块数
        num_blocks_height = height // self.subrows
        num_blocks_width = width // self.subcoloums
        self.rowlast=num_blocks_height
        self.last=num_blocks_width
        # 定义一个函数来处理每个块
        def process_block(block):
            # 获取块的形状
            block_height, block_width = block.shape
            # 创建一个 16x16 的张量，填充 -1
            padded_block = torch.full(( self.subrows, self.subcoloums), -1, dtype=block.dtype)
            # 将原始块复制到填充的张量中
            padded_block[:block_height, :block_width] = block
            return padded_block
        # 创建一个空列表来存储分割后的块
        split_blocks = []
        # 遍历每个块
        for i in range(num_blocks_height ):
            for j in range(num_blocks_width ):
                # 计算块的起始和结束索引
                start_row = i *  self.subrows
                end_row = (i + 1) *  self.subrows
                start_col = j * self.subcoloums
                end_col = (j + 1) * self.subcoloums
                # 获取块
                block = self.index[start_row:end_row, start_col:end_col]
                # 处理块并添加到列表中
                split_blocks.append(process_block(block))
        return split_blocks
    
    def calculate (self):
        split_blocks=self.split_tensor()
        print(1)
        cycle=0
        not_finsh=1
        i=0
        j=0
        while(i!=len(split_blocks)) :
            sub_block=split_blocks[i]
            load_cycle=torch.sum(torch.tensor([
                self.spad.load(val) for val in sub_block.flatten()
            ], dtype=torch.int32))
            if i % self.rowlast==0 :
                comput_cycle=32
            else:
                comput_cycle=16
            print(load_cycle)
            i=i+1
        return cycle
       
         # for i in range(len(split_blocks)):

        #     if i % self.rowlast==0 :

        #     #检查是否边界
        #     else : 
        #         if  self.read_ready==1:  #检查数据是否有效

        #         else:
          
        #     #计算

In [38]:
class cache_sim:
    def __init__(self, mode, linesize, cachesize, num_lin, codebook):
        self.linesize = linesize
        self.num_lin = num_lin
        self.cache = torch.full((num_lin, linesize), -1)
        self.codebook = codebook
        
        # 预建 codebook 值到行的快速映射 (值 -> 行索引)
        self.codebook_value_map = {}
        for row_idx, row in enumerate(codebook):
            for val in row:
                val = val.item()
                if val not in self.codebook_value_map:
                    self.codebook_value_map[val] = row_idx
        
        # 使用 OrderedDict 实现高效 LRU 机制
        self.lru = OrderedDict()
        
        # 值到缓存行的反向映射 (值 -> 所在行集合)
        self.value_to_lines = defaultdict(set)
        # 缓存行到值的反向映射 (行 -> 存储的值集合)
        self.line_to_values = [set() for _ in range(num_lin)]

    def maintain_record(self, line):
        """ 更新 LRU 记录，O(1) 时间复杂度 """
        if line in self.lru:
            self.lru.move_to_end(line)
        else:
            self.lru[line] = None

    def is_empty(self):
        """ 返回可替换的行或 -1 """
        if len(self.lru) < self.num_lin:
            return -1
        else:
            # 弹出 LRU 行并清理其反向映射
            lru_line, _ = self.lru.popitem(last=False)
            for val in self.line_to_values[lru_line]:
                self.value_to_lines[val].remove(lru_line)
                if not self.value_to_lines[val]:
                    del self.value_to_lines[val]
            self.line_to_values[lru_line].clear()
            return lru_line

    def load_data(self, data):
        """ 加载数据到缓存，向量化操作 """
        data_val = data.item()
        if data_val not in self.codebook_value_map:
            raise ValueError(f"Value {data_val} not in codebook")
        
        code_row = self.codebook[self.codebook_value_map[data_val]]
        target_line = self.is_empty()
        
        if target_line == -1:
            # 寻找未使用的行
            for line in range(self.num_lin):
                if line not in self.lru:
                    target_line = line
                    break
            # 如果所有行已使用，触发 LRU 替换
            if target_line == -1:
                target_line = self.is_empty()
        # print(target_line)
        # 更新缓存行数据
        self.cache[target_line] = code_row
        # 更新反向映射
        new_values = set(code_row.tolist())
        for val in new_values:
            self.value_to_lines[val].add(target_line)
        self.line_to_values[target_line] = new_values
        self.maintain_record(target_line)

    def sim(self, read_data):
        """ 批量查询优化，O(1) 时间复杂度 """
        data_val = read_data.item()
        
        # 
        if data_val in self.value_to_lines:
            # 取第一个关联行并更新 LRU
            line = next(iter(self.value_to_lines[data_val]))
            self.maintain_record(line)
            return True
        else:
            self.load_data(read_data)
            # print('not hit')
            return False
        
class spad:
    def __init__(self, num_bank,num_rows,l1cache,l2cache,code):
       self.numbank=num_bank
       self.numrows=num_rows
       self.rowsize=2*16 #一个row存储16个数据 每个数据2B
       self.realtime_numrow=0
       self.cache1=l1cache
       self.cache2=l2cache
       self.l1hit=0
       self.l2hit=0
    def check(self):
        if self.realtime_numrow>1:
            self.realtime_numrow=self.realtime_numrow-2
            return True
        else :
            return False
    def load(self,index):
        l1=self.cache1.sim(index)
        l2=self.cache2.sim(index)
        self.realtime_numrow=self.realtime_numrow+1 
        if l1==True:
            self.l1hit=self.l1hit+1
            return 2
        elif l1==False and l2==True:
            self.l2hit=self.l2hit+1
            return 10
        elif l1==False and l2 ==False:
            return 40
        else :
            return False
    def show_hit(self):
        print("l1_hit:",self.l1hit)
        print("l2_hit:",self.l2hit)

    

In [40]:
cache1=cache_sim(mode=None, linesize=4, cachesize=None, num_lin=256, codebook=code)
cache2=cache_sim(mode=None, linesize=4, cachesize=None, num_lin=1024, codebook=code)
sim_spad=spad(4,256,cache1,cache2,None)
sim_pe=PE(None,recon_weight,2,16,sim_spad)
print(sim_pe.calculate())